In [1]:
import baseline

In [2]:
X, y = baseline.baselineXy()

In [3]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import QuantileTransformer, KBinsDiscretizer, Normalizer, PowerTransformer, RobustScaler, SplineTransformer, StandardScaler, MinMaxScaler
td = {
 'sdoif_mean_id': QuantileTransformer,
 'sdoif_mean_sq': RobustScaler,
 'elevation_mean_sqrt': QuantileTransformer,
 'procurv_mean_id': StandardScaler,
 'placurv_mean_id': Normalizer,
 'lsfactor_mean_id': Normalizer,
 'slope_mean_id': RobustScaler,
 'twi_mean_id': Normalizer,
 'twi_mean_sqrt': QuantileTransformer,
 'aspect_mean_id': QuantileTransformer
}

In [4]:
for key, t in td.items():
    X[key] = t().fit_transform(X[[key]])
X = X.to_numpy()

In [5]:
groups = [
    [1, 2, 6, 7],
    [3, 8],
    [4, 5, 9, 10],
    [11, 12],
    [13],
    [14, 15],
    [16, 17, 21, 22],
    [18, 23],
    [19, 20, 24, 25],
    list(range(1, 26))
]

def get_group_mean(df, col, group):
    cols = [f"{i}_{col}" for i in group]
    return df[cols].to_numpy().mean(axis=1)

In [6]:
import baseline
X, y = baseline.originalXy()
get_group_mean(X, "slope", groups[0])

array([37.73571  , 31.7261425, 49.1132025, ..., 39.9281625, 35.24104  ,
       20.02481  ])

In [7]:
import pandas as pd
import numpy as np
grid_groups = pd.DataFrame()

def normalize(x):
    return (x - x.min()) / (x.max() - x.min())

def special_normalize(x):
    return ((x - x.min()) / (x.max() - x.min()) + 0.01) / 1.01

fns = [(np.array, 'id'), (np.sqrt, 'sqrt'), (np.square, 'sq'), (np.log, 'log')]
print(len(baseline.continuous) * len(groups) * len(fns))
for col in baseline.continuous:
    for i, group in enumerate(groups):
        for fn, name in fns:
            this = f"{col}_g{i}_{name}"
            if name == 'log':
                grid_groups[this] = fn(special_normalize(get_group_mean(X, col, group)))
            else:
                grid_groups[this] = fn(normalize(get_group_mean(X, col, group)))

320


/var/folders/cj/7rdlf_rs5k55gkh65d8gkkd00000gn/T/ipykernel_9582/391222228.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grid_groups[this] = fn(normalize(get_group_mean(X, col, group)))
/var/folders/cj/7rdlf_rs5k55gkh65d8gkkd00000gn/T/ipykernel_9582/391222228.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grid_groups[this] = fn(normalize(get_group_mean(X, col, group)))
/var/folders/cj/7rdlf_rs5k55gkh65d8gkkd00000gn/T/ipykernel_9582/391222228.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually 

In [8]:
from finn.feat_trans import test_feats
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.linear_model import LogisticRegression

# estimators = [
#     ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
#     ('svr', make_pipeline(StandardScaler(),
#                           LinearSVC(random_state=42)))
# ]
models = [LogisticRegression, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, CatBoostClassifier]

"""
k = 3
kf = KFold(n_splits=k)
for m in models:
    f1m = 0
    for train_ind, test_ind in kf.split(grid_groups.to_numpy()):
        X_train, X_test = grid_groups.to_numpy()[train_ind], grid_groups.to_numpy()[test_ind]
        y_train, y_test = y[train_ind], y[test_ind]
        X_train, y_train = SMOTE().fit_resample(X_train, y_train)
        mym = m().fit(X_train, y_train)
        probs = mym.predict_proba(X_test)[:, 1]
        y_pred = np.array(probs > 0.4, dtype=int)
        #print(y_pred)
        f1 = f1_score(y_test, y_pred)
        print(f1)
        f1m += f1 / k
    print(m.__name__, f1m)
"""

'\nk = 3\nkf = KFold(n_splits=k)\nfor m in models:\n    f1m = 0\n    for train_ind, test_ind in kf.split(grid_groups.to_numpy()):\n        X_train, X_test = grid_groups.to_numpy()[train_ind], grid_groups.to_numpy()[test_ind]\n        y_train, y_test = y[train_ind], y[test_ind]\n        X_train, y_train = SMOTE().fit_resample(X_train, y_train)\n        mym = m().fit(X_train, y_train)\n        probs = mym.predict_proba(X_test)[:, 1]\n        y_pred = np.array(probs > 0.4, dtype=int)\n        #print(y_pred)\n        f1 = f1_score(y_test, y_pred)\n        print(f1)\n        f1m += f1 / k\n    print(m.__name__, f1m)\n'

In [9]:
grid_groups

,sdoif_g0_id,sdoif_g0_sqrt,sdoif_g0_sq,sdoif_g0_log,sdoif_g1_id,sdoif_g1_sqrt,sdoif_g1_sq,sdoif_g1_log,sdoif_g2_id,sdoif_g2_sqrt,...,aspect_g7_sq,aspect_g7_log,aspect_g8_id,aspect_g8_sqrt,aspect_g8_sq,aspect_g8_log,aspect_g9_id,aspect_g9_sqrt,aspect_g9_sq,aspect_g9_log
0,0.680410,0.824870,0.462958,-0.380419,0.680235,0.824763,0.462719,-0.380674,0.680087,0.824674,...,0.102406,-1.118586,0.323977,0.569190,0.104961,-1.106634,0.303978,0.551342,0.092402,-1.168384
1,0.960227,0.979912,0.922036,-0.040175,0.960051,0.979822,0.921698,-0.040357,0.959901,0.979745,...,0.254094,-0.675332,0.481153,0.693652,0.231508,-0.720950,0.500208,0.707254,0.250208,-0.682887
2,0.979967,0.989933,0.960336,-0.020034,0.980065,0.989982,0.960528,-0.019935,0.980189,0.990045,...,0.516833,-0.326154,0.722408,0.849946,0.521873,-0.321368,0.790661,0.889191,0.625145,-0.232268
3,0.029984,0.173158,0.000899,-3.229237,0.030168,0.173691,0.000910,-3.224625,0.030344,0.174196,...,0.178926,-0.846976,0.410311,0.640555,0.168355,-0.876712,0.466744,0.683187,0.217850,-0.750726
4,0.690175,0.830768,0.476342,-0.366375,0.689855,0.830575,0.475900,-0.366832,0.689496,0.830359,...,0.125539,-1.019687,0.329545,0.574060,0.108600,-1.090100,0.329019,0.573602,0.108253,-1.091650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,0.646353,0.803961,0.417773,-0.431006,0.646819,0.804251,0.418375,-0.430297,0.647233,0.804508,...,0.108934,-1.088610,0.294239,0.542438,0.086576,-1.199894,0.308574,0.555495,0.095218,-1.153850
10860,0.857721,0.926132,0.735685,-0.151835,0.857601,0.926067,0.735479,-0.151974,0.857472,0.925998,...,0.158778,-0.905288,0.481153,0.693652,0.231508,-0.720950,0.401679,0.633782,0.161346,-0.897461
10861,0.739961,0.860210,0.547543,-0.297684,0.739836,0.860137,0.547358,-0.297851,0.739731,0.860076,...,0.154014,-0.920145,0.364499,0.603738,0.132860,-0.992116,0.360548,0.600456,0.129995,-1.002723
10862,0.944510,0.971859,0.892099,-0.056508,0.944649,0.971930,0.892361,-0.056362,0.944817,0.972017,...,0.523297,-0.320025,0.709069,0.842063,0.502780,-0.339748,0.698702,0.835884,0.488185,-0.354270


In [10]:
from sklearn.feature_selection import SelectFromModel
def tree_features(X, n):
    clf = ExtraTreesClassifier(n_estimators=50)
    clf = clf.fit(X, y)
    model = SelectFromModel(clf, prefit=True, max_features=n)

    X_clf = model.transform(X)
    return test_feats(ExtraTreesClassifier(n_estimators=50), X_clf, y)

In [11]:
gg = grid_groups.to_numpy()
hX = StandardScaler().fit_transform(gg)
#, QuantileTransformer().fit_transform(gg), Normalizer().fit_transform(gg)]

In [12]:
tree_features(hX, None)

0.6153780884089599

In [13]:
def something():
    for sc in [StandardScaler, MinMaxScaler, QuantileTransformer, PowerTransformer]:
        hX = sc().fit_transform(gg)
        clf = ExtraTreesClassifier()
        clf = clf.fit(hX, y)
        model = SelectFromModel(clf, prefit=True)
        X_clf = model.transform(hX)
        print(sc.__name__, test_feats(ExtraTreesClassifier(n_estimators=50), X_clf, y))

In [14]:
import lightgbm as lgb

In [16]:
gg = gg[:, :50]
gg.shape

(10864, 50)

In [17]:
kf = KFold(n_splits=3)
f1m = 0
gg = grid_groups.to_numpy()
for train_ind, test_ind in kf.split(gg):
    X_train, X_test = gg[train_ind], gg[test_ind]
    y_train, y_test = y[train_ind], y[test_ind]
    X_train, y_train = SMOTE().fit_resample(X_train, y_train)
    X_train, X_test = StandardScaler().fit_transform(X_train), StandardScaler().fit_transform(X_test)
    ds = lgb.Dataset(X_train, label=y_train)
    bst = lgb.train({'num_leaves': 30, 'objective': 'binary', 'metric': 'auc', 'njobs': 2}, ds)
    #y_pred = bst.predict(X_test)
    #print(y_pred)
    #f1 = f1_score(y_test, y_pred)
    #print(f1)
    #f1m += f1 / 3
print(f1m)

In [ ]:
gg.shape